# Azure email extraction

## Define variables

In [1]:
num_gen_rand = 73883472 # Put YOUR randomly generated number here

laquelle_cred = 'default'  # default, cli
RESOURCE_GROUP_NAME = f"gdr-name-{num_gen_rand}"
subscription_id = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"
STORAGE_ACCOUNT_NAME = f"storagename{num_gen_rand}"
laquelle_conteneur = 'blobcont'
local_path = "/.../Azure_cogservices_Formrecognizer/data0"
FORMREC_KEY = 'a4fc6b77f9ff448496300bcd40a612af'


## Create Azure objects with bash CLI

Run the create_resources.sh script to create the needed Azure objects.

chmod +x create_resources.sh  (Make it executable, if it is not)

./create_resources.sh  (Run script)

## Create Azure objects and get object information with Python SDK

In [51]:
val = 'X0'
if val == 'X0':
    from azure.identity import DefaultAzureCredential
    credential = DefaultAzureCredential()
else:
    print('Do not create credentials')

val = 'X0'
if val == 'X0':
    # -------------------------------------
    # Get storage account information
    # -------------------------------------
    import os
    from azure.mgmt.storage import StorageManagementClient

    # Provision the storage account, starting with a management object
    storage_client = StorageManagementClient(credential, subscription_id)

    # Step 3: Retrieve the account's primary access key and generate a connection string.
    keys = storage_client.storage_accounts.list_keys(RESOURCE_GROUP_NAME, STORAGE_ACCOUNT_NAME)

    print(f"Primary key for storage account: {keys.keys[0].value}")

    # Verifier acces
    os.environ['AZURE_STORAGE_CONNECTION_STRING'] = f"DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={keys.keys[0].value}"

    print(f"Connection string: {os.getenv('AZURE_STORAGE_CONNECTION_STRING')}")
else:
    print('Do not createstockage')

val = 'X0'
if val == 'X0':
    if laquelle_conteneur == 'blobcont':
        # ---------------
        # Blob container
        # ---------------
        # Create a container blob in the storage account
        CONTAINER_NAME = f"blobcontainer-{num_gen_rand}-python2"

        # Create blobstorage client: Client help you upload files to container
        from azure.storage.blob import BlobServiceClient
        conn_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

        # Create the BlobServiceClient object: this is a credential object
        blob_service_client = BlobServiceClient.from_connection_string(conn_str)
        print('blob_service_client: ', blob_service_client)

        # Create the blob storage container
        from azure.storage.blob import PublicAccess
        container_client = blob_service_client.create_container(CONTAINER_NAME, 
                                                                public_access='BLOB')
    else:
        # ---------------
        # Container only
        # ---------------
        from azure.storage.blob import ContainerClient
        CONTAINER_NAME = f"container-{num_gen_rand}-python1"
        container_client = ContainerClient.from_connection_string(conn_str=conn_str,
                                                                  container_name=CONTAINER_NAME)
else:
    print('Do not create a  container')

    
    
# --------------------------------------- 
# Upload the created file
# ---------------------------------------
val = 'X0'
if val == 'X0':
    upload_file_path = [local_path + '/' + f'{filename}' for filename in os.listdir(local_path)]
    
    # Upload chaque image
    from azure.storage.blob import BlobClient
    from azure.storage.blob import ContentSettings

    for chaquefichierPATH in upload_file_path:
        # Create a blob client using the local file name as the name for the blob
        filenom = chaquefichierPATH.split(local_path+'/')[1]

        blob = BlobClient.from_connection_string(conn_str=conn_str,
                                             container_name=CONTAINER_NAME,
                                             blob_name=filenom)

        # Content-type
        with open(file=chaquefichierPATH, mode="rb") as data:
            blob.upload_blob(data, 
                             overwrite=True, 
                             content_settings=ContentSettings(content_type='image/pdf'))
else:
    print('Do not upload documents')

# --------------------------------------- 
# Form recognizer
# --------------------------------------- 
endpoint = "https://global.api.cognitive.microsoft.com/"

blobs_list = container_client.list_blobs()
for blob in blobs_list:
    print(blob.name + '\n')
    
    # Obtain the URL for each pdf
    url = f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{CONTAINER_NAME}/{blob.name}"
    
    text = analyze_read(url, endpoint, FORMREC_KEY)

Ne crée pas credentials
Ne crée pas stockage
Ne crée pas conteneur
ContactUsUpwork.pdf

Document contains content:  Contact Us | Upwork
___
Upwork
About Us Our Leaders
https://www.upwork.com/about/contact
Find Talent > Find Work Why Upwork
Enterprise
Q 1
Search
1 sur 3
Careers
Board of Directors
Press
Contact us
Contact us
Reach out anytime
Customer Support
Visit
Help Center
Enterprise Solutions
866.262.4478
Press Inquiries
press@upwork.com -
-
Our Offices
Global HQ 475 Brannan St. Suite 430 San Francisco CA 94107 Phone: (650) 316-7500
y
8
a
10/05/2023 14:50
----Key-value pairs found in document----
Document contains handwritten content
Document contains handwritten content
----Analyzing Read from page #1----
Page has width: 8.2778 and height: 11.6944, measured with unit: inch
...Line # 0 has text content 'Contact Us | Upwork' within bounding box '[0.1654, 0.169], [1.3325, 0.169], [1.3325, 0.2951], [0.1654, 0.2951]'
...Line # 1 has text content '___' within bounding box '[0.4999, 0.658

## Subfunctions

In [46]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient


def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])


def analyze_read(url, endpoint, key):
    
    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    
    poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-read", url)
    result = poller.result()

    print ("Document contains content: ", result.content)
    
    # General documents
    print("----Key-value pairs found in document----")
    for kv_pair in result.key_value_pairs:
        if kv_pair.key and kv_pair.value:
            print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
        else:
            print("Key '{}': Value:".format(kv_pair.key.content))
    
    # Layout
    for table_idx, table in enumerate(result.tables):
        print(
            "Table # {} has {} rows and {} columns".format(
            table_idx, table.row_count, table.column_count
            )
        )

        for cell in table.cells:
            print(
                "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content.encode("utf-8"),
                )
            )
    
    # Read
    for idx, style in enumerate(result.styles):
        print(
            "Document contains {} content".format(
                "handwritten" if style.is_handwritten else "no handwritten"
            )
        )

    for page in result.pages:
        print("----Analyzing Read from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            print(
                "...Line # {} has text content '{}' within bounding box '{}'".format(
                    line_idx,
                    line.content,
                    format_bounding_box(line.polygon),
                )
            )
        
        
        for word in page.words:
            print("...Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )

    print("----------------------------------------")
    return result.content

#if __name__ == "__main__":
#    analyze_read()


In [ ]:
text

In [39]:
def mpf(arr):
    to_remove = ['[', ',', ']', "'"]
    txt = str(arr)
    for i in to_remove:
        txt = txt.replace(i, "" )
    txt = txt.split(' ')
    
    arr = [i for i in txt if i != '']
    return arr


In [40]:
def emails(nested_str_arr):

    # Fonction que recherche des emails address
    import re
    match_object = []
    for ind, text_str in enumerate(nested_str_arr):
        resp = re.match(r'(\w+)@(\w+)\.(\w+)', text_str)
        if resp != None:
            match_object.append(resp)
    return match_object


In [54]:
# Regarder au text
out = text.split("\n")
out = [i.split(' ') for i in out]
out1 = mpf(out)
emails_out = emails(out1)
print('emails_out: ', emails_out)


emails_out:  [<re.Match object; span=(0, 16), match='press@upwork.com'>]


## Clean UP

Run the clean_resources.sh script.

chmod +x clean_resources.sh  (Make it executable, if it is not)

./clean_resources.sh  (Run script)